In [1]:
from numpy import *
 
import convert
from plotting.draw import *  # noqa
import src.bins as bins  # noqa
from src.dut_analysis import DUTAnalysis, Analysis
from src.batch_analysis import BatchAnalysis
from src.run import load_nrs
from src.scan import Ensemble, Scan, VScan, TScan
from utility.utils import *  # noqa
from functools import partial

Welcome to JupyROOT 6.26/10


## Parameters

In [2]:
run = 485
verbose=0
test=0
testcampaign='201912'
batch='v1'
dut=0
do_convert=0
remove_meta=0
runplan=None

## Main analysis

In [3]:
if runplan is not None:
    from src.spreadsheet import make
    make(runplan)
    exit(2)

ensembles = load_json(Ensemble.FilePath)

if run in ensembles:
    s = VScan if 'v-' in run else TScan if 't-' in run else Scan
    z = s(run, verbose, test)

else:
    ana = Analysis(testcampaign)
    runs = load_nrs(ana.BeamTest.Path)
    is_batch = not (run in runs and batch is None)
    dut_ana = partial(BatchAnalysis, choose(batch, run)) if is_batch else partial(DUTAnalysis, run)
    dut_ana = partial(dut_ana, dut, testcampaign)

    if is_batch:
        bc = convert.BatchConvert(dut_ana.args[0], dut_ana.args[-1], verbose=False, force=False)
        if do_convert:
            remove_file(bc.Batch.FileName)
            bc.remove_aux_files()
        if not bc.Batch.FileName.exists() and not test:
            bc.run()

    if remove_meta:
        z = dut_ana(verbose=False, test=True)
        z.remove_metadata()

    if do_convert and not is_batch:
        z = dut_ana(verbose=False, test=True)
        z.remove_file()
        z.Converter.remove_aux_files()

    z = dut_ana(verbose=verbose, test=test)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
STARTING BATCH ANALYSIS of D02, batch v1 (Dec 2019), 18.80M ev |
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



In [4]:
%jsroot on

In [5]:
z.draw_charge_distribution()
get_last_canvas().Draw()

INFO:     16:26:23 --> saving plot: SignalDist

INFO:     16:26:23 --> wrote file /home/dmitry/mounts/high-rate/content/duts/D02/201912/b-v1/SignalDist.html
INFO:     16:26:23 --> https://diamond.ethz.ch/high-rate/content/duts/D02/201912/b-v1/SignalDist.html


### realign before plotting in pixel

In [6]:
z.Residuals.align()

array([[ 9.99999837e-01,  3.80373872e-04,  4.14372117e-02],
       [-8.55841212e-04,  9.99999837e-01, -7.54905347e-02],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [7]:
z.draw_cs_in_pixel()
get_last_canvas().Draw()

INFO:     16:26:24 --> saving plot: CsInPixel

INFO:     16:26:24 --> wrote file /home/dmitry/mounts/high-rate/content/duts/D02/201912/b-v1/CsInPixel.html
INFO:     16:26:24 --> https://diamond.ethz.ch/high-rate/content/duts/D02/201912/b-v1/CsInPixel.html


In [8]:
p = z.Draw.ResultsDir

In [9]:
list(p.rglob("*"))

[PosixPath('/home/dmitry/software/HighResAnalysis/results/201912/SignalDist.pdf'),
 PosixPath('/home/dmitry/software/HighResAnalysis/results/201912/CsInPixel.pdf')]